Source: https://www.daniellowengrub.com/blog/2024/01/03/fully-homomorphic-encryption

# LWE 
#### Preliminaries
Let $q \in \mathbb{N}$. Denote the integers modulo $q$ as $\mathbb{Z}_q:=\mathbb{Z}/q\mathbb{Z}$. For convenience, we set
$$q=2^{32}$$
We will use the **symmetric residue elements**, i.e.
$$\mathbb{Z}/q\mathbb{Z} = [-2^{31}, 2^{31})$$

#### LWE  Definitions
Let 
- $\boldsymbol{s} \in \mathbb{Z}_{q}^n$ be a vector of length $n$ with elements in $\mathbb{Z}_{q}$ **(Secret Vector)**
- $\boldsymbol{a}_{1},\dots, \boldsymbol{a}_{m} \in \mathbb{Z}_{q}^m$ be **random vectors**
- $e_{1}, \dots, e_{m} \in \mathbb{Z}_{q}$ be **noise** drawn from some distribution $\chi$ 
- $b_{i}=\boldsymbol{a}_{i}^T \boldsymbol{s} + e_{i}$ be the **noisy dot-product**

The general problem of LWE can be formulated as

---
Given  $m$ random vectors $\boldsymbol{a}_{1}, \dots, \boldsymbol{a}_{m}$ and the noisy dot products $\boldsymbol{b}_{1}, \dots, \boldsymbol{b}_{m}$, is it possible to efficiently learn the secret vector $\boldsymbol{s}$?

---

#### The Noise Distribution
To create the errors, we draw real numbers from a **Gaussian distribution** $\mathcal{N}(0, \sigma)$ with $\sigma \ll 1$, such that
$$-\frac{1}{2} \leq x_{i} < \frac{1}{2}$$
Then
$$e = \left\lfloor  x \cdot \frac{q}{2}  \right\rfloor $$
The given distribution is denoted as $\mathcal{N}_{q}(0, \sigma)$. 

#### Typical Values
$$\begin{align}
q &= 2^{32} \\
n &= 500 \\
\sigma &= 2^{-20}
\end{align}$$

#### Building the Encryption Scheme
- Keygeneration:
	$$\boldsymbol{s} \in_{R} \{ 0,1 \}^n \subset \mathbb{Z}_{q}^n$$
	where $\in_{R}$ denotes random selection
- Encryption Function $$\text{Enc}_{\boldsymbol{s}}(m):=(\boldsymbol{a}, \boldsymbol{a}^T \boldsymbol{s} + m + e) \in \mathbb{Z}_{q}^n \times \mathbb{Z}_{q}$$
- Decryption Function $$\begin{align}
\text{Dec}_{\boldsymbol{s}}(\boldsymbol{a},b) &= b - \boldsymbol{a}^T \boldsymbol{s} \\
&= (\boldsymbol{a}^T\boldsymbol{s}+m+e) - \boldsymbol{a}^T \boldsymbol{s} \\
&= m+e
\end{align}$$
where we still need to remove the error by some method later described
	

In [21]:
from typing import Optional
import numpy as np

INT32_MIN = np.iinfo(np.int32).min
INT32_MAX = np.iinfo(np.int32).max


def uniform_sample_int32(size: int) -> np.ndarray:
    return np.random.randint(
        low=INT32_MIN,
        high=INT32_MAX + 1,
        size=size,
        dtype=np.int32,
    )


def gaussian_sample_int32(std: float, size: Optional[float]) -> np.ndarray:
    return np.int32(INT32_MAX * np.random.normal(loc=0.0, scale=std, size=size))

print(INT32_MIN)
print(INT32_MAX)
print(uniform_sample_int32(1))
print(gaussian_sample_int32(0.01, 1))

-2147483648
2147483647
[-817160971]
[12797299]


In [ ]:
import dataclasses
import numpy as np

@dataclasses.dataclass
class LweConfig:
    # Size of the LWE encryption key.
    dimension: int

    # Standard deviation of the encryption noise.
    noise_std: float

@dataclasses.dataclass
class LwePlaintext:
    message: np.int32


@dataclasses.dataclass
class LweCiphertext:
    config: LweConfig
    a: np.ndarray  # An int32 array of size config.dimension
    b: np.int32


@dataclasses.dataclass
class LweEncryptionKey:
    config: LweConfig
    key: np.ndarray  # An int32 array of size config.dimension


def generate_lwe_key(config: LweConfig) -> LweEncryptionKey:
    return LweEncryptionKey(
        config=config,
        key=np.random.randint(
            low=0, high=2, size=(config.dimension,), dtype=np.int32
        ),
    )


def lwe_encrypt(
    plaintext: LwePlaintext, key: LweEncryptionKey
) -> LweCiphertext:
    a = uniform_sample_int32(size=key.config.dimension)
    noise = gaussian_sample_int32(std=key.config.noise_std, size=None)

    # b = (a, key) + message + noise
    b = np.add(np.dot(a, key.key), plaintext.message, dtype=np.int32)
    b = np.add(b, noise, dtype=np.int32)

    return LweCiphertext(config=key.config, a=a, b=b)


def lwe_decrypt(
    ciphertext: LweCiphertext, key: LweEncryptionKey
) -> LwePlaintext:
    return LwePlaintext(
        np.subtract(ciphertext.b, np.dot(ciphertext.a, key.key), dtype=np.int32)
    )

In [28]:
# Lattice Estimator https://github.com/malb/lattice-estimator

LWE_CONFIG = LweConfig(dimension=1024, noise_std=2**(-24))

# Generate an LWE key.
key = generate_lwe_key(LWE_CONFIG)

# This is the plaintext that we will encrypt.
plaintext = LwePlaintext(2**29)

# Encrypt the plaintext 1000 times and store the error of each ciphertext.
errors = []
for _ in range(1000):
    ciphertext = lwe_encrypt(plaintext, key)
    errors.append(lwe_decrypt(ciphertext, key).message - plaintext.message)

In [32]:
from pprint import pprint
pprint(key.key)

array([0, 1, 0, ..., 1, 1, 1], dtype=int32)
